# Implementation of a Blind Signature Scheme with RSA

## RSA Key Pair Generation

In [26]:
import random

# Miller-Rabin Primality Test
def miller_rabin(n, k=5):  # number of rounds k=5 gives a good balance between accuracy and performance
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False
    
    # Write n-1 as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

# Function to generate a large prime number
def generate_large_prime(bits=32):
    while True:
        p = random.getrandbits(bits)
        if p % 2 != 0 and miller_rabin(p):
            return p

In [27]:
p = generate_large_prime() # prime number
q = generate_large_prime() # prime number

n = p*q # prime composite
print('Prime composite n:',n)

phi_n = (p-1)*(q-1) # Euler Totient Function of n
print('Euler totient function of n:',phi_n)

Prime composite n: 3595555571165719001
Euler totient function of n: 3595555567191036900


### Greatest Common Divisor utility function

In [28]:
def gcd(a, b):
    if((a<0) or (b<0) or (a<b)):
        print("wrong parameter input")
        return

    while(b != 0):
        r = a % b
        a = b
        b = r
        
    return a

In [29]:
e = generate_large_prime() # public exponent
print((1 < e) and (e < phi_n))
print(gcd(phi_n,e))
print('Public exponent:',e)

True
1
Public exponent: 592285037


### Modulo Inverse utility function

In [30]:
# Calculate the modular inverse of k mod q efficiently

def extended_gcd(k, q):
    if k == 0:
        return q, 0, 1
    gcd, x1, y1 = extended_gcd(q % k, k)
    x = y1 - (q // k) * x1
    y = x1
    return gcd, x, y

def mod_inverse(k, q):
    gcd, invk, _ = extended_gcd(k, q)
    if gcd != 1:
        raise ValueError(f"Inverse doesn't exist for k = {k} mod q = {q}, as they are not coprime.")
    return invk % q

In [31]:
d = mod_inverse(e,phi_n)
print('Secret exponent:',d)

Secret exponent: 1224645852219829373


In [32]:
print('Secret key (d):',d)
print('Public key (e,n):',e,n)

Secret key (d): 1224645852219829373
Public key (e,n): 592285037 3595555571165719001


In [33]:
import hashlib

# get the message as input from a message.txt file
def read_message_from_file(file_name, n):
    with open(file_name, 'r') as file:
        # Read the message from the file
        message = file.read()
        print('message:', message)
        # Convert the message to a sha256 hash
        sha256_hash = hashlib.sha256(message.encode()).hexdigest()
        
    return int(sha256_hash, 16) % n # Convert the hash to an integer

# write the signature on the disk
def write_signature_to_file(signature, file_name):
    with open(file_name, 'w') as file:
        file.write(str(signature))
     
# read the signature from the disk        
def read_signature_from_file(file_name):
    with open(file_name, 'r') as file:
        signature = int(file.read())
        
    return signature
    
message_filename = 'message.txt'
signature_filename = 'signature.txt'
m = read_message_from_file(message_filename, n)

message: Ado mama Anjula


## Verifying Key Pair using RSA Public Key Encryption

In [34]:
print('Plaintext :',m)
c = pow(m, e, n) # public key encryption
print('Ciphertext:',c)
m1 = pow(c, d, n) # public key decryption
print('Decryption:',m1)

Plaintext : 1469870244539900414
Ciphertext: 78136843000316676
Decryption: 1469870244539900414


## Blind Signature Generation

In [35]:
k = generate_large_prime() # blinding factor
bf = pow(k, e, n) # blinding factor
print('Blinding factor:',bf)
m1 = (m*bf)%n # blinded message
print('Blinded message:',m1)

Blinding factor: 3306608607340632204
Blinded message: 364403294961421539


In [36]:
s1 = pow(m1, d, n) # signature on blinded message
print('Signature on blinded message:',s1)
write_signature_to_file(s1, signature_filename)

Signature on blinded message: 3422019689197458095


In [37]:
s1 = read_signature_from_file(signature_filename)
invk = mod_inverse(k,n)
s = (s1*invk)%n # recovery of signature
print('Signature on original message:',s)

s0 = pow(m, d, n) # compute the signature directly
print('Signature on original message when directly computed:',s0)

Signature on original message: 1343383836941302309
Signature on original message when directly computed: 1343383836941302309


In [38]:
# check if the signature is valid
if s == s0:
    print('Signature is valid')
else:
    print('Signature is invalid')

Signature is valid
